# Generate data for LSTM Model

## Importing library

In [4]:
import cv2
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import mediapipe as mp
from sklearn.model_selection import train_test_split
import copy
from concurrent.futures import ThreadPoolExecutor
import random

## Hyperparameters

In [5]:
# DATA
BATCH_SIZE = 32
AUTO = tf.data.AUTOTUNE
INPUT_SHAPE = (30, 1086)
NUM_CLASSES = 40

# DATA PATH - GOOGLE DRIVE PATH
VIDEO_DATASET_PATH = "Videos"
LSTM_DATASET_PATH = "LSTM_DATASET"

# MEDIAPIPE HOLISTIC
POSE_NUM_LANDMARK = 33
LEFT_HAND_NUM_LANDMARK = 21
RIGHT_HAND_NUM_LANDMARK = 21
FACE_NUM_LANDMARK = 468


## Function to get mediapipe coordinate

In [6]:
mp_holistic = mp.solutions.holistic

def apply_mediapipe_holistic(frame):
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        # Convert the frame to RGB as Mediapipe Holistic requires RGB input
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(image=frame_rgb)

        # Initialize list to store the coordinates
        coordinates = []

        # Pose landmark
        if results.pose_landmarks is not None:
            for landmark in results.pose_landmarks.landmark:
                coordinates.extend([landmark.x, landmark.y])
        else:
            # Add zeros for the pose landmarks if nothing is detected
            num_pose_landmarks = POSE_NUM_LANDMARK * 2
            coordinates.extend([0.0] * num_pose_landmarks)

        # Left-hand landmark
        if results.left_hand_landmarks is not None:
            for landmark in results.left_hand_landmarks.landmark:
                coordinates.extend([landmark.x, landmark.y])
        else:
            # Add zeros for the left-hand landmarks if nothing is detected
            num_left_hand_landmarks = LEFT_HAND_NUM_LANDMARK * 2
            coordinates.extend([0.0] * num_left_hand_landmarks)

        # Right-hand landmark
        if results.right_hand_landmarks is not None:
            for landmark in results.right_hand_landmarks.landmark:
                coordinates.extend([landmark.x, landmark.y])
        else:
            # Add zeros for the right-hand landmarks if nothing is detected
            num_right_hand_landmarks = RIGHT_HAND_NUM_LANDMARK * 2
            coordinates.extend([0.0] * num_right_hand_landmarks)

        # Face landmark
        if results.face_landmarks is not None:
            for landmark in results.face_landmarks.landmark:
                coordinates.extend([landmark.x, landmark.y])
        else:
            # Add zeros for the face landmarks if nothing is detected
            num_face_landmarks = FACE_NUM_LANDMARK * 2
            coordinates.extend([0.0] * num_face_landmarks)

        return np.array(coordinates)

## Extract frame from video and covert to coordinate

In [7]:
def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    required_frames = 30

    # Calculate the frame interval to
    # exactly 30 frames
    frame_interval = max(1, total_frames // required_frames)

    for frame_idx in range(0, total_frames, frame_interval):
        if len(frames) >= required_frames:
            break

        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if not ret:
            break

        # Apply mediapipe_holistic
        coordinates = apply_mediapipe_holistic(frame)
        frames.append(coordinates)

    cap.release()
    return np.array(frames)

## Create data - test

In [5]:
# Ensure the new dataset directory exists
if not os.path.exists(LSTM_DATASET_PATH):
    os.makedirs(LSTM_DATASET_PATH)

# Iterate through each class folder in the original dataset
class_folders = os.listdir(VIDEO_DATASET_PATH)
for class_folder in class_folders:
    class_dir = os.path.join(VIDEO_DATASET_PATH, class_folder)

    # Ensure it's a directory
    if os.path.isdir(class_dir):
        # Create a new folder for the class in the new dataset directory
        new_class_dir = os.path.join(LSTM_DATASET_PATH, class_folder)
        os.makedirs(new_class_dir, exist_ok=True)

        # Get the list of video files in the class folder
        video_files = [file for file in os.listdir(class_dir) if file.endswith(".mp4")]

        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(extract_frames, os.path.join(class_dir, video_file))
                           for video_file in video_files]

            for future, video_file in zip(futures, video_files):
                frames = future.result()

                # Save the frames as a numpy array in the new dataset directory
                np.save(os.path.join(new_class_dir, f"{video_file.split('.')[0]}.npy"), frames)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
